<a href="https://colab.research.google.com/github/Asaf95/Apple-stock-price-prediction/blob/Einav/Apple_Stock_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing modules

In [1]:
 pip install alpha_vantage

     |████████████████████████████████| 1.1 MB 6.6 MB/s 
     |████████████████████████████████| 144 kB 56.6 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 271 kB 41.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import stml_m3
import get_data
#import data_preparation
import lstm_model
import data_pre
#import aap_02 

# Main

In [ ]:
import pandas as pd

df = pd.read_csv("/content/rawdata_AAPL.csv")
#df = pd.read_csv(data, encoding = "ISO-8859-1")
df = pd.DataFrame(df)
df_newL = df.copy()
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-03-22,120.330002,123.870003,120.260002,123.389999,122.660965,111912300
1,2021-03-23,123.330002,124.239998,122.139999,122.540001,121.815987,95467100
2,2021-03-24,122.820000,122.900002,120.070000,120.089996,119.380455,88530500
3,2021-03-25,119.540001,121.660004,119.000000,120.589996,119.877502,98844700
4,2021-03-26,120.349998,121.480003,118.919998,121.209999,120.493843,94071200
...,...,...,...,...,...,...,...
249,2022-03-16,157.050003,160.000000,154.460007,159.589996,159.589996,102300200
250,2022-03-17,158.610001,161.000000,157.630005,160.619995,160.619995,75615400
251,2022-03-18,160.509995,164.479996,159.759995,163.979996,163.979996,123351200
252,2022-03-21,163.509995,166.350006,163.009995,165.380005,165.380005,95722200


LSTM Model

In [ ]:
import datetime
def str_to_datetime(s):
    split = s.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return datetime.datetime(year=year, month=month, day=day)
    
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = str_to_datetime(first_date_str)
    last_date  = str_to_datetime(last_date_str)

    target_date = first_date
    
    dates = []
    X, Y = [], []

    last_time = False
    while True:
      df_subset = dataframe.loc[:target_date].tail(n+1)
      
      if len(df_subset) != n+1:
        print(f'Error: Window of size {n} is too large for date {target_date}')
        return

In [ ]:
def New_LSTM(df, start_time, end_time):
  df_newL = df
  df_newL = df_newL[['Date', 'Close']]
  df_newL['Date']
  import datetime

  datetime_object = str_to_datetime('1986-03-19')
  datetime_object
  df_newL['Date'] = df_newL['Date'].apply(str_to_datetime)
  df_newL['Date']
  df_newL.index = df_newL.pop('Date')
  df_newL

  import matplotlib.pyplot as plt

  plt.plot(df_newL.index, df_newL['Close'])
  import numpy as np

  def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = str_to_datetime(first_date_str)
    last_date  = str_to_datetime(last_date_str)

    target_date = first_date
    
    dates = []
    X, Y = [], []

    last_time = False
    while True:
      df_subset = dataframe.loc[:target_date].tail(n+1)
      
      if len(df_subset) != n+1:
        print(f'Error: Window of size {n} is too large for date {target_date}')
        return

      values = df_subset['Close'].to_numpy()
      x, y = values[:-1], values[-1]

      dates.append(target_date)
      X.append(x)
      Y.append(y)

      next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
      next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
      next_date_str = next_datetime_str.split('T')[0]
      year_month_day = next_date_str.split('-')
      year, month, day = year_month_day
      next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))
      
      if last_time:
        break
      
      target_date = next_date

      if target_date == last_date:
        last_time = True
      
    ret_df = pd.DataFrame({})
    ret_df['Target Date'] = dates
    
    X = np.array(X)
    for i in range(0, n):
      X[:, i]
      ret_df[f'Target-{n-i}'] = X[:, i]
    
    ret_df['Target'] = Y

    return ret_df

  # Start day second time around: '2021-03-25'
  windowed_df = df_to_windowed_df(df_newL, 
                                  start_time, 
                                  end_time,
                                  n=3)
  print(windowed_df)

  def windowed_df_to_date_X_y(windowed_dataframe):
    df_as_np = windowed_dataframe.to_numpy()

    dates = df_as_np[:, 0]

    middle_matrix = df_as_np[:, 1:-1]
    X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

    Y = df_as_np[:, -1]
    return dates, X.astype(np.float32), Y.astype(np.float32)

  dates, X, y = windowed_df_to_date_X_y(windowed_df)

  dates.shape, X.shape, y.shape

  q_80 = int(len(dates) * .8)
  q_90 = int(len(dates) * .9)

  dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

  dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
  dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

  plt.plot(dates_train, y_train)
  plt.plot(dates_val, y_val)
  plt.plot(dates_test, y_test)

  plt.legend(['Train', 'Validation', 'Test'])
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras import layers

  model = Sequential([layers.Input((3, 1)),
                      layers.LSTM(64),
                      layers.Dense(32, activation='relu'),
                      layers.Dense(32, activation='relu'),
                      layers.Dense(1)])

  model.compile(loss='mse', 
                optimizer=Adam(learning_rate=0.001),
                metrics=['mean_absolute_error'])

  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
  train_predictions = model.predict(X_train).flatten()

  # Showing the results of the model 

  plt.plot(dates_train, train_predictions)
  plt.plot(dates_train, y_train)
  plt.legend(['Training Predictions', 'Training Observations'])

  val_predictions = model.predict(X_val).flatten()

  plt.plot(dates_val, val_predictions)
  plt.plot(dates_val, y_val)
  plt.legend(['Validation Predictions', 'Validation Observations'])

  test_predictions = model.predict(X_test).flatten()

  plt.plot(dates_test, test_predictions)
  plt.plot(dates_test, y_test)
  plt.legend(['Testing Predictions', 'Testing Observations'])
  plt.plot(dates_train, train_predictions)
  plt.plot(dates_train, y_train)
  plt.plot(dates_val, val_predictions)
  plt.plot(dates_val, y_val)
  plt.plot(dates_test, test_predictions)
  plt.plot(dates_test, y_test)
  plt.legend(['Training Predictions', 
              'Training Observations',
              'Validation Predictions', 
              'Validation Observations',
              'Testing Predictions', 
              'Testing Observations'])
  
  from copy import deepcopy

  recursive_predictions = []
  recursive_dates = np.concatenate([dates_val, dates_test])

  for target_date in recursive_dates:
    last_window = deepcopy(X_train[-1])
    next_prediction = model.predict(np.array([last_window])).flatten()
    recursive_predictions.append(next_prediction)
    last_window[-1] = next_prediction

  plt.plot(dates_train, train_predictions)
  plt.plot(dates_train, y_train)
  plt.plot(dates_val, val_predictions)
  plt.plot(dates_val, y_val)
  plt.plot(dates_test, test_predictions)
  plt.plot(dates_test, y_test)
  plt.plot(recursive_dates, recursive_predictions)
  plt.legend(['Training Predictions', 
              'Training Observations',
              'Validation Predictions', 
              'Validation Observations',
              'Testing Predictions', 
              'Testing Observations',
              'Recursive Predictions'])

In [ ]:

#import pandas as pd

"""
the main module calls all the other modules and have access to all the functions that are needed 
for building the stock model
the main three modules are:
1. get_data()- using this module for getting the last stock price and the history prices.
2. data_preparation()- using this module to prepare the data for the ML models by fixing the types of columns 
fixing the stock splits issue and more.
3. model_lstm()- predict the stock value and is the main ML model in this project.
"""
"""
def main():
  # geting the AAPL stock data using API's
  #get_data.get_apple_stock_data()

  # clean_df = data_pre.basic_prepare(df)

  # reading the raw data that was created by the API's 
  df = pd.read_csv("/content/rawdata_AAPL.csv")
  df_clean = data_pre.basic_prepare(df)
  df_newL = df_clean.copy()
  df_clean = df_clean.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low',
                            'close': 'Close', 'volume': 'Volume','date':'Date'})
  df_n = df_clean.copy()
  df_n = df_n.iloc[::-1]
  #New_LSTM(df_n,'2009-03-25', '2022-03-21')
  print(df_n)

  reg_results = aap_02.aapl_reg_models (df_n)
  print(reg_results)
  
    #get_data.get_apple_stock_data()
    # raw_data = pd.read_csv('data/apple_daily_data')
    # prepare_data = data_preparation.basic_prepare(raw_data)
    # recursive_predictions = model_lstm.LSTM_prepare_variables(prepare_data)
    # print(recursive_predictions)
    #results = ml_classification_models.classification_models(prepare_data)
    #print(results)


if __name__ == "__main__":
  main()
"""


#Regression models

NameError: ignored